In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# List of tech stock symbols
tech_stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NFLX', 'NVDA', 'AMD', 'SSNLF', 'CSCO', 'ORCL', 'SAP', 'IBM', 'INTC', 'CRM', 'ADBE']
START_DATE = '2018-01-01'
END_DATE = '2024-05-05'

def fetch_stock_data(symbol, start_date, end_date):
    df = yf.download(symbol, start=start_date, end=end_date)
    df['Symbol'] = symbol
    return df

def load_data():
    combined_df = pd.DataFrame()
    for stock in tech_stocks:
        df = fetch_stock_data(stock, START_DATE, END_DATE)
        combined_df = pd.concat([combined_df, df])
    return combined_df

combined_df = load_data()

# Calculate daily returns
combined_df['Daily Return'] = combined_df.groupby('Symbol')['Close'].pct_change()

# Calculate daily price change
combined_df['Daily Change'] = combined_df['Close'] - combined_df['Open']

# Calculate monthly average returns
combined_df['Month'] = combined_df.index.to_period('M')
monthly_avg_returns = combined_df.groupby(['Month', 'Symbol'])['Daily Return'].mean().reset_index()
monthly_avg_returns['Month'] = monthly_avg_returns['Month'].astype(str)  # Convert Period to string

# Calculate annual volatility
combined_df['Year'] = combined_df.index.year
annual_volatility = combined_df.groupby(['Year', 'Symbol'])['Daily Return'].std().reset_index()

# Correlation Matrix of Daily Returns
correlation_matrix = combined_df.pivot_table(values='Daily Return', index='Date', columns='Symbol').corr()

# Moving Averages: 30 days and 90 days
for symbol in combined_df['Symbol'].unique():
    combined_df.loc[combined_df['Symbol'] == symbol, 'MA30'] = combined_df[combined_df['Symbol'] == symbol]['Close'].rolling(window=30).mean()
    combined_df.loc[combined_df['Symbol'] == symbol, 'MA90'] = combined_df[combined_df['Symbol'] == symbol]['Close'].rolling(window=90).mean()

# Volatility Analysis: Calculate rolling standard deviation of daily returns
combined_df['Volatility'] = combined_df.groupby('Symbol')['Daily Return'].rolling(window=30).std().reset_index(0, drop=True)

# Calculate Relative Strength Index (RSI)
def calculate_rsi(data, window):
    diff = data.diff(1).dropna()
    gain = diff.where(diff > 0, 0)
    loss = -diff.where(diff < 0, 0)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

for symbol in combined_df['Symbol'].unique():
    combined_df.loc[combined_df['Symbol'] == symbol, 'RSI'] = calculate_rsi(combined_df[combined_df['Symbol'] == symbol]['Close'], window=14)

# Calculate Bollinger Bands
def calculate_bollinger_bands(data, window):
    rolling_mean = data.rolling(window=window).mean()
    rolling_std = data.rolling(window=window).std()
    upper_band = rolling_mean + (rolling_std * 2)
    lower_band = rolling_mean - (rolling_std * 2)
    return rolling_mean, upper_band, lower_band

for symbol in combined_df['Symbol'].unique():
    rolling_mean, upper_band, lower_band = calculate_bollinger_bands(combined_df[combined_df['Symbol'] == symbol]['Close'], window=20)
    combined_df.loc[combined_df['Symbol'] == symbol, 'Bollinger_Middle'] = rolling_mean
    combined_df.loc[combined_df['Symbol'] == symbol, 'Bollinger_Upper'] = upper_band
    combined_df.loc[combined_df['Symbol'] == symbol, 'Bollinger_Lower'] = lower_band

# Calculate Cumulative Returns
combined_df['Cumulative Return'] = combined_df.groupby('Symbol')['Daily Return'].cumsum()

# Calculate Drawdowns
def calculate_drawdown(data):
    cumulative = (1 + data).cumprod()
    peak = cumulative.cummax()
    drawdown = (cumulative - peak) / peak
    return drawdown

for symbol in combined_df['Symbol'].unique():
    combined_df.loc[combined_df['Symbol'] == symbol, 'Drawdown'] = calculate_drawdown(combined_df[combined_df['Symbol'] == symbol]['Daily Return'])

# Plotting Daily Returns
fig = px.line(combined_df.reset_index(), x='Date', y='Daily Return', color='Symbol', 
              title='Daily Returns of Tech Stocks', color_discrete_sequence=px.colors.qualitative.Alphabet)
fig.show()

# Plotting Correlation Matrix
fig = px.imshow(correlation_matrix, text_auto=True, title='Correlation Matrix of Daily Returns', labels=dict(color="Correlation"))
fig.show()

# Plotting Moving Averages
fig = go.Figure()
colors = {symbol: px.colors.qualitative.Alphabet[i % len(px.colors.qualitative.Alphabet)] for i, symbol in enumerate(combined_df['Symbol'].unique())}
for symbol in combined_df['Symbol'].unique():
    symbol_data = combined_df[combined_df['Symbol'] == symbol]
    fig.add_trace(go.Scatter(x=symbol_data.index, y=symbol_data['MA30'], name=f'{symbol} 30-Day MA', line=dict(color=colors[symbol])))
    fig.add_trace(go.Scatter(x=symbol_data.index, y=symbol_data['MA90'], name=f'{symbol} 90-Day MA', visible='legendonly', line=dict(color=colors[symbol], dash='dash')))
fig.update_layout(title='Moving Averages for Tech Stocks', xaxis_title='Date', yaxis_title='Moving Average Price')
fig.show()


# Plotting Volatility
fig = px.line(combined_df.reset_index(), x='Date', y='Volatility', color='Symbol', 
              title='30-Day Rolling Volatility of Tech Stocks', color_discrete_sequence=px.colors.qualitative.Alphabet)
fig.show()

# Scatter plot of Volume vs. Closing Price
fig = px.scatter(combined_df.reset_index(), x='Close', y='Volume', color='Symbol', 
                 title='Volume vs. Closing Price of Stocks', color_discrete_sequence=px.colors.qualitative.Alphabet)
fig.update_layout(xaxis_title='Close Price', yaxis_title='Volume')
fig.show()


# Daily Change Analysis
fig = px.box(combined_df, x='Symbol', y='Daily Change', title='Daily Change in Price (Close - Open) by Stock', color='Symbol', color_discrete_sequence=px.colors.qualitative.Alphabet)
fig.show()

# Monthly Average Returns
fig = px.line(monthly_avg_returns, x='Month', y='Daily Return', color='Symbol', 
              title='Monthly Average Returns of Tech Stocks', color_discrete_sequence=px.colors.qualitative.Alphabet)
fig.show()

# Annual Volatility
fig = px.bar(annual_volatility, x='Year', y='Daily Return', color='Symbol', 
             title='Annual Volatility of Tech Stocks', barmode='group', color_discrete_sequence=px.colors.qualitative.Alphabet)
fig.show()

# Candlestick Chart for each stock
for symbol in combined_df['Symbol'].unique():
    symbol_data = combined_df[combined_df['Symbol'] == symbol]
    fig = go.Figure(data=[go.Candlestick(x=symbol_data.index,
                                         open=symbol_data['Open'],
                                         high=symbol_data['High'],
                                         low=symbol_data['Low'],
                                         close=symbol_data['Close'])])
    fig.update_layout(title=f'Candlestick Chart for {symbol}', xaxis_title='Date', yaxis_title='Price')
    fig.show()

# Scatter plot of Opening vs. Closing Prices
fig = px.scatter(combined_df, x='Open', y='Close', color='Symbol', 
                 title='Opening vs. Closing Prices of Tech Stocks', color_discrete_sequence=px.colors.qualitative.Alphabet)
fig.show()

# Plotting RSI
fig = px.line(combined_df.reset_index(), x='Date', y='RSI', color='Symbol', 
              title='Relative Strength Index (RSI) of Tech Stocks', color_discrete_sequence=px.colors.qualitative.Alphabet)
fig.show()

# Plotting Bollinger Bands
fig = go.Figure()
for symbol in combined_df['Symbol'].unique():
    symbol_data = combined_df[combined_df['Symbol'] == symbol]
    fig.add_trace(go.Scatter(x=symbol_data.index, y=symbol_data['Bollinger_Middle'], name=f'{symbol} Bollinger Middle', line=dict(color=colors[symbol])))
    fig.add_trace(go.Scatter(x=symbol_data.index, y=symbol_data['Bollinger_Upper'], name=f'{symbol} Bollinger Upper', line=dict(color=colors[symbol], dash='dash')))
    fig.add_trace(go.Scatter(x=symbol_data.index, y=symbol_data['Bollinger_Lower'], name=f'{symbol} Bollinger Lower', line=dict(color=colors[symbol], dash='dash')))
fig.update_layout(title='Bollinger Bands for Tech Stocks', xaxis_title='Date', yaxis_title='Price')
fig.show()

# Plotting Cumulative Returns
fig = px.line(combined_df.reset_index(), x='Date', y='Cumulative Return', color='Symbol', 
              title='Cumulative Returns of Tech Stocks', color_discrete_sequence=px.colors.qualitative.Alphabet)
fig.show()

# Plotting Drawdowns
fig = px.line(combined_df.reset_index(), x='Date', y='Drawdown', color='Symbol', 
              title='Drawdowns of Tech Stocks', color_discrete_sequence=px.colors.qualitative.Alphabet)
fig.show()

# Save updated DataFrame to CSV (optional)
combined_df.to_csv('updated_combined_tech_stocks_data.csv')

print("Data fetching completed and combined CSV saved as 'updated_combined_tech_stocks_data.csv'.")


ModuleNotFoundError: No module named 'yfinance'